In [1]:
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

import chromedriver_autoinstaller

import time
from datetime import datetime

import re
import os
import pandas as pd
import json
import emoji

import traceback

from IPython.display import clear_output

chromedriver_autoinstaller.install()

def remove_emoji(text):
    for x in emoji.emoji_list(text)[::-1]:
        text = text[:x['match_start']]+text[x['match_end']:]
    return text

In [2]:
file_list = [x for x in os.listdir(os.getcwd()+'/csv') if '.csv' in x]
file_list

['KAKAO_luxury.csv',
 'KAKAO_touched.csv',
 'KAKAO_baby.csv',
 'INFO_KAKAO_small_luxury.csv',
 'KAKAO_funny.csv',
 'ERR_2022-11-27 22:50:45.630008.csv',
 'INFO_KAKAO_samll_gift.csv',
 'ERR_2022-11-27 22:58:37.124221.csv',
 'INFO_KAKAO_health.csv',
 'INFO_KAKAO_touched.csv',
 'INFO_KAKAO_luxury.csv',
 'KAKAO_samll_gift.csv',
 'INFO_KAKAO_birthday.csv',
 'ERR_2022-11-27 22:58:47.857406.csv',
 'KAKAO_small_luxury.csv',
 'KAKAO_birthday.csv',
 'KAKAO_health.csv',
 'INFO_KAKAO_funny.csv',
 'INFO_KAKAO_baby.csv']

In [3]:
file_list = ['KAKAO_birthday.csv']

In [4]:
KAKAO_SEARCH_URL = 'https://gift.kakao.com/search'

In [5]:
process_list = []

In [7]:
EXCEPT_PRODUCT = ['원소주 스피릿']

In [8]:
%%time
for file in file_list:
    df = pd.read_csv(f'./csv/{file}',index_col=0)
    driver = webdriver.Chrome()
    for idx in df.index:
        if idx%25==0:
            clear_output(wait=True)
            print(f'{file}-{idx}')
            print((df.isna().sum()/(len(df.index)) * 100))
        try:
            driver.get(KAKAO_SEARCH_URL)
            time.sleep(1)
            
            name = remove_emoji(df.loc[idx,'name'])
            if name in EXCEPT_PRODUCT:
                continue
            
            search_box = driver.find_element(By.XPATH,'//*[@id="searchInput"]')
            search_box.send_keys(name+Keys.RETURN)
            time.sleep(4)

            result_box = driver.find_element(By.CLASS_NAME, 'thumb_prd')
            result_box.click()
            time.sleep(3)

            detail_button = driver.find_element(By.XPATH, '//*[@id="mArticle"]/app-home/div/app-contents/app-product-detail-tab/div/ul/li[3]/a/span')
            detail_button.click()
            
            time.sleep(2)
            
            detail_table = driver.find_element(By.TAG_NAME,'table')
            detail_str = detail_table.get_attribute('innerHTML')
            detail_list = [x for x in re.findall('>(.*?)<',detail_str) if x]
            detail_dict = dict(zip(detail_list[1::2],detail_list[2::2]))

            for key, val in detail_dict.items():
                df.loc[idx,key] = val
        except Exception as e:
            print(f'{name}')
            print(e)
            driver.quit()
            df.to_csv(f'./ERR_{datetime.now()}.csv')
            break
    df.to_csv(f'./INFO_{file}')
    process_list.append(f'{file}->{datetime.now()}')
    

KAKAO_birthday.csv-75
name                                          0.000000
price                                         0.000000
type                                          0.000000
용량 또는 중량                                     81.420765
제품 주요 사양                                     81.420765
                                               ...    
색상                                           99.453552
제품구성                                         99.453552
세탁방법/취급시 주의사항                                99.453552
법에 의한 인증, 허가 등을 받았음을 확인할 수 있는 경우 그에 대한 사항    99.453552
A/S 책임자와 전화번호 또는 소비자상담 관련 전화번호               99.453552
Length: 73, dtype: float64
WON SHOT SET - 원소주 스피릿(1ea)&니트잔(2ea)
Alert Text: 로그인이 필요한 메뉴입니다.
로그인하시겠습니까?
Message: unexpected alert open: {Alert text : 로그인이 필요한 메뉴입니다.
로그인하시겠습니까?}
  (Session info: chrome=107.0.5304.110)
Stacktrace:
0   chromedriver                        0x0000000107cd2938 chromedriver + 4745528
1   chromedriver                        0x0000000107c54443 chr

In [66]:
process_list

['KAKAO_luxury.csv->2022-11-27 20:21:47.308876',
 'KAKAO_touched.csv->2022-11-27 20:56:46.923172',
 'KAKAO_baby.csv->2022-11-27 21:14:01.150252',
 'KAKAO_funny.csv->2022-11-27 22:02:46.490822',
 'KAKAO_samll_gift.csv->2022-11-27 22:50:45.650641',
 'KAKAO_small_luxury.csv->2022-11-27 22:58:37.139755',
 'KAKAO_birthday.csv->2022-11-27 22:58:47.861670',
 'KAKAO_health.csv->2022-11-27 23:33:42.754910']